## Download magnet
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
save_path = "B\u1ED9 nh\u1EDB Colab" #@param ["","Google Drive","Bộ nhớ Colab"]
if save_path == "Google Drive":
  params = {"save_path": "/content/drive/MyDrive"}
else:
  params = {"save_path": "/content/"}
while True:
    magnet_link = input("Enter Magnet Link Or Type e To Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


In [ ]:
#@title **Download link file trực tiếp về Google Drive hoặc Colab (Thử nghiệm)**
import json
import requests
import gdown
from urllib.parse import urlparse as urlp

#@markdown **Kiểu link**
server = "Direct link" #@param ["","Google Drive","OneDrive (Sharepoint)", "Dropbox","helurl.com","Direct link"]

#@markdown **Nhập link**

#@markdown Với kiểu link Google Drive thì hãy chỉ điền id file vào mục bên dưới

#@markdown Ví dụ với 1 đường link Google Drive cơ bản sẽ trông như sau: https://drive.google.com/file/d/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx/view

#@markdown Đoạn "xxxxx" đó chính là id file.
link = "https://c6.kemono.su/data/e1/5f/e15fbfdf572658a401f241a8b7d08aedf9f229ad07fc343a7371605ebd920b1a.mp4?f=%E5%AE%8C%E6%88%90%E5%8B%95%E7%94%BB_Fantia.mp4" #@param {type:"string"}

#@markdown **Vị trí lưu**
vi_tri_luu ="B\u1ED9 nh\u1EDB Colab" #@param ["","Google Drive","Bộ nhớ Colab"]

#@markdown **(Nâng cao) --no-check-certificate**

#@markdown Đôi lúc 1 số link trực tiếp không thể xác nhận được chứng chỉ, tùy chọn này giúp ta bỏ qua việc xác nhận chứng chỉ và tiếp tục tải file.

#@markdown Mặc định là tắt.
no_check_certificate = False #@param {type:"boolean"}

#@markdown Chỉ định sẵn tên file, nhớ kèm phần mở rộng file.

#@markdown Với kiểu link Google Drive thì mục này phải bắt buộc bật và bạn phải đặt tên file
custom_name = True #@param {type:"boolean"}
name = "\u5B8C\u6210\u52D5\u753B_Fantia.mp4" #@param {type:"string"}

def onedrive(link, name, custom_name, vi_tri_luu):

  def onedown(link, name, vi_tri_luu):
    if vi_tri_luu == "Google Drive":
      !wget -O "$name" -P /content/drive/MyDrive $link
    else:
      !wget -O "$name"  $link

  x = link.split("?")
  link = x[0] + "?download=1"

  if custom_name:
    onedown(link, name, vi_tri_luu)
  else:
    name = input("Nhập tên file (kèm phần mở rộng): ")
    onedown(link, name, vi_tri_luu)

def dropbox(link, name, custom_name, vi_tri_luu):

  def dropdown(link, name, vi_tri_luu):
    if vi_tri_luu == "Google Drive":
      !wget -O "$name" -P /content/drive/MyDrive $link
    else:
      !wget -O "$name"  $link

  if custom_name:
      dropdown(link, name, vi_tri_luu)
  else:
      link = link.replace("dl=0", "dl=1")
      path = urlp(link).path.split('/')
      name = path[len(path)-1]
      name = name.replace("%20","_")
      name = name.replace("%28","(")
      name = name.replace("%29",")")
      dropdown(link, name, vi_tri_luu)

def helurl(link, name, custom_name, vi_tri_luu):

  def helurldown(link, name, vi_tri_luu):
    if vi_tri_luu == "Google Drive":
      !wget -O "$name" -P /content/drive/MyDrive $link
    else:
      !wget -O "$name" $link

  if (link.find("api") < 0):
      x = link.split("/")
      h = x[len(x)-1]
      url = 'https://helurl.com/api/v1/shareable-links/'+ h + '?withEntries=true&page=1&order=updated_at:desc'
      x = requests.get(url)
      data = x.json()
      id = data['link']['id']
      hash = data['link']['entry']['hash']
      link = 'https://helurl.com/api/v1/file-entries/download/'+ hash +'?shareable_link=' + str(id) + '&password=null'#;
      print("Direct link: ",link)#;
      if custom_name:
        helurldown(link, name, vi_tri_luu)
      else:
        name = data['link']['entry']['name']
        helurldown(link, name, vi_tri_luu)

  else:
    if custom_name:
      helurldown(link, name, custom_name, vi_tri_luu)
    else:
      name = input('Link bạn nhập là link trực tiếp từ helurl, nhập tên file bạn muốn lưu: ')
      helurldown(link, name, custom_name, vi_tri_luu)

def direct(link, name, custom_name, vi_tri_luu, no_check_certificate):

  def directdown(link, name, vi_tri_luu, no_check_certificate):
    if no_check_certificate:
      if vi_tri_luu == "Google Drive":
        !wget -O "$name" -P /content/drive/MyDrive $link --no-check-certificate
      else:
        !wget -O "$name" $link --no-check-certificate
    else:
      !wget -O "$name" $link

  if custom_name:
    directdown(link, name, vi_tri_luu, no_check_certificate)
  else:
    path = urlp(link).path.split('/')
    name = path[len(path)-1]
    name = name.replace("%20","_")
    name = name.replace("%28","(")
    name = name.replace("%29",")")
    directdown(link, name, vi_tri_luu, no_check_certificate)

def gdrive(link, name, vi_tri_luu):
  #def gdown(link, name, vi_tri_luu):
    #!gdown -O "$name"
  FILEID = f'https://docs.google.com/uc?export=download&id={link}'
  FILENAME = name
  if vi_tri_luu == "Google Drive":
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "$FILEID" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$link" -O "$FILENAME" && rm -rf /tmp/cookies.txt -P /content/drive/MyDrive
  else:
    #!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILEID" -O FILENAME && rm -rf /tmp/cookies.txt
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "$FILEID" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$link" -O "$FILENAME" && rm -rf /tmp/cookies.txt

if server == "OneDrive (Sharepoint)":
  onedrive(link, name, custom_name, vi_tri_luu)
if server == "Dropbox":
  dropbox(link, name, custom_name, vi_tri_luu)
if server == "helurl.com":
  helurl(link, name, custom_name, vi_tri_luu)
if server == "Google Drive":
  gdrive(link, name, vi_tri_luu)
if server == "Direct link":
  direct(link, name, custom_name, vi_tri_luu, no_check_certificate)